## nesting in a SequentialActivity

This notebook provides an example of OpenCLSim's capability to nest various processes in a __SequentialActivity__.

#### 0. Import libraries

In [1]:
import datetime, time
import simpy

import pandas as pd

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

#### 1. Initialise simpy environment

In [2]:
# setup environment (simulation time needs to match the available weather data)
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)

#### 2. Define object classes
In this example we won't use object classes

#### 3. Create objects
##### 3.1. Create site object(s)
No site objects are created

##### 3.2. Create vessel object(s)
No vessel objects are created

##### 3.3 Create activity/activities

In [3]:
# initialise registry
registry = {}

In [4]:
a = model.BasicActivity(
    env= my_env,
    name= "a",
    ID= "a",
    registry= registry,
    duration= 1,
)
a2 = model.BasicActivity(
    env= my_env,
    name= "a2",
    ID= "a2",
    registry= registry,
    duration= 1,
)

Sa=model.SequentialActivity(    
    env= my_env,
    name= "Sa",
    ID= "Sa",
    registry= registry,
    sub_processes= [a, a2],
)

c = model.BasicActivity(
    env= my_env,
    name= "c",
    ID= "c",
    registry= registry,
    duration= 1,
)

Sc=model.SequentialActivity(    
    env= my_env,
    name= "Sc",
    ID= "Sc",
    registry= registry,
    sub_processes= [c, Sa],
)

b = model.BasicActivity(
    env= my_env,
    name= "b",
    ID= "b",
    registry= registry,
    duration= 10,
    start_event=[
        {
            "name": "a",
            "type": "activity",
            "state": "done"
        }
    ]

)

Sb=model.SequentialActivity(    
    env= my_env,
    name= "Sb",
    ID= "Sb",
    registry= registry,
    sub_processes= [b],
)

#### 4. Register processes and run simpy

In [5]:
model.register_processes([Sc,Sb])
my_env.run()

#### 5. Inspect results
##### 5.1 Inspect logs

In [6]:
display(pd.DataFrame(Sc.log))

display(pd.DataFrame(c.log))
display(pd.DataFrame(Sa.log))

display(pd.DataFrame(a.log))
display(pd.DataFrame(a2.log))

display(pd.DataFrame(b.log))

,Timestamp,ActivityID,ActivityState,ObjectState,ActivityLabel
0,1970-01-01 00:00:00,Sc,START,{},{}
1,1970-01-01 00:00:00,Sc,START,{},"{'type': 'subprocess', 'ref': 'c'}"
2,1970-01-01 00:00:01,Sc,STOP,{},"{'type': 'subprocess', 'ref': 'c'}"
3,1970-01-01 00:00:01,Sc,START,{},"{'type': 'subprocess', 'ref': 'Sa'}"
4,1970-01-01 00:00:03,Sc,STOP,{},"{'type': 'subprocess', 'ref': 'Sa'}"
5,1970-01-01 00:00:03,Sc,STOP,{},{}


,Timestamp,ActivityID,ActivityState,ObjectState,ActivityLabel
0,1970-01-01 00:00:00,c,START,{},{}
1,1970-01-01 00:00:01,c,STOP,{},{}


,Timestamp,ActivityID,ActivityState,ObjectState,ActivityLabel
0,1970-01-01 00:00:01,Sa,START,{},{}
1,1970-01-01 00:00:01,Sa,START,{},"{'type': 'subprocess', 'ref': 'a'}"
2,1970-01-01 00:00:02,Sa,STOP,{},"{'type': 'subprocess', 'ref': 'a'}"
3,1970-01-01 00:00:02,Sa,START,{},"{'type': 'subprocess', 'ref': 'a2'}"
4,1970-01-01 00:00:03,Sa,STOP,{},"{'type': 'subprocess', 'ref': 'a2'}"
5,1970-01-01 00:00:03,Sa,STOP,{},{}


,Timestamp,ActivityID,ActivityState,ObjectState,ActivityLabel
0,1970-01-01 00:00:01,a,START,{},{}
1,1970-01-01 00:00:02,a,STOP,{},{}


,Timestamp,ActivityID,ActivityState,ObjectState,ActivityLabel
0,1970-01-01 00:00:02,a2,START,{},{}
1,1970-01-01 00:00:03,a2,STOP,{},{}


,Timestamp,ActivityID,ActivityState,ObjectState,ActivityLabel
0,1970-01-01 00:00:00,b,WAIT_START,{},{}
1,1970-01-01 00:00:02,b,WAIT_STOP,{},{}
2,1970-01-01 00:00:02,b,START,{},{}
3,1970-01-01 00:00:12,b,STOP,{},{}


##### 5.2 Visualise gantt charts

In [7]:
plot.get_gantt_chart([Sc,c,Sa,a,a2,b])